# Graph Puzzle

The following graph puzzle is entirely a classical computational problem, but has relevance to parts of quantum error correction.
This notebook describes the problem in a slightly simplified way without using any confusing quantum information language.


## Outline of the puzzle

Consider a periodic graph defined as coordinates over edges (`e`) and vertices (`V`) that are arranged in a `D` by `D` grid structure like so:

In [65]:
from src.classical.periodic_grid_graph import PeriodicGridGraph

graph = PeriodicGridGraph(4)
graph.draw_graph()


        e0              e1              e2              e3      


e4      V0      e5      V1      e6      V2      e7      V3      e4      


        e8              e9              e10             e11     


e12     V4      e13     V5      e14     V6      e15     V7      e12     


        e16             e17             e18             e19     


e20     V8      e21     V9      e22     V10     e23     V11     e20     


        e24             e25             e26             e27     


e28     V12     e29     V13     e30     V14     e31     V15     e28     


        e0              e1              e2              e3      




D = 4 repeating graph



This example has a dimension of `D = 4`, and indexes run from left-to-right, top-to-bottom, for both edges and vertices.

The graph has periodic boundary conditions, where the left/right and top/bottom set of edges are identical. In other words, the graph "wraps around" like a pacman grid - if you travel beyond the top of the grid you will re-emerge at the bottom, and vice versa. Same goes for left/right.

You can see here that every edge connects 2 adjacent vertices, and every vertex has a degree of 4 (i.e. is incident to 4 edges).

Here are the rules of the graph:

- both edges and vertices can exist in one of two states: `0` or `1`;
- if an edge is in the state `1`, then it flips the state of its 2 adjacent vertices;

Let's take a look at the same graph again, where all edges and vertices are in the `0` state:

In [66]:
graph.draw_graph()


        e0              e1              e2              e3      


e4      V0      e5      V1      e6      V2      e7      V3      e4      


        e8              e9              e10             e11     


e12     V4      e13     V5      e14     V6      e15     V7      e12     


        e16             e17             e18             e19     


e20     V8      e21     V9      e22     V10     e23     V11     e20     


        e24             e25             e26             e27     


e28     V12     e29     V13     e30     V14     e31     V15     e28     


        e0              e1              e2              e3      




D = 4 repeating graph



Let's see what happens when we flip the state of edge `e17`:

In [67]:
edges_to_flip = [17]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)



        e0              e1              e2              e3      


e4      V0      e5      V1      e6      V2      e7      V3      e4      


        e8              e9              e10             e11     


e12     V4      e13     V5      e14     V6      e15     V7      e12     


        e16             e17             e18             e19     


e20     V8      e21     V9      e22     V10     e23     V11     e20     


        e24             e25             e26             e27     


e28     V12     e29     V13     e30     V14     e31     V15     e28     


        e0              e1              e2              e3      




D = 4 repeating graph



We see here that by flipping edge `e17` we have also flipped the state of vertices `V5` and `V9` from `0` to `1`.

What happens if we also flip edge `e22`? Let's have a look:

In [68]:
edges_to_flip = [17, 22]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)


        e0              e1              e2              e3      


e4      V0      e5      V1      e6      V2      e7      V3      e4      


        e8              e9              e10             e11     


e12     V4      e13     V5      e14     V6      e15     V7      e12     


        e16             e17             e18             e19     


e20     V8      e21     V9      e22     V10     e23     V11     e20     


        e24             e25             e26             e27     


e28     V12     e29     V13     e30     V14     e31     V15     e28     


        e0              e1              e2              e3      




D = 4 repeating graph



We see here that by flipping edges `e17` and `e22` we have also flipped the state of vertices `V5` and `V10` from `0` to `1`.

But since both `e17` and `e22` have each flipped the state of `V9`, that vertex goes back to the original `0` state.
This introduces the concept of the flipped edge "chain": given a chain of adjacent flipped edges, only vertices at the ends of the chain will be flipped.

What happens when we flip the states of edges `e14` and `e18`?

In [69]:
edges_to_flip = [14, 18]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)


        e0              e1              e2              e3      


e4      V0      e5      V1      e6      V2      e7      V3      e4      


        e8              e9              e10             e11     


e12     V4      e13     V5      e14     V6      e15     V7      e12     


        e16             e17             e18             e19     


e20     V8      e21     V9      e22     V10     e23     V11     e20     


        e24             e25             e26             e27     


e28     V12     e29     V13     e30     V14     e31     V15     e28     


        e0              e1              e2              e3      




D = 4 repeating graph



We see that this has resulted in flipping the exact same vertices that edges `e17` and `e22` did!

A similar thing can happen if the flipped edge chain crosses one of the boundaries.
Consider what happens when we flip edges `e20` and `e21`:

In [70]:
edges_to_flip = [20, 21]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)


        e0              e1              e2              e3      


e4      V0      e5      V1      e6      V2      e7      V3      e4      


        e8              e9              e10             e11     


e12     V4      e13     V5      e14     V6      e15     V7      e12     


        e16             e17             e18             e19     


e20     V8      e21     V9      e22     V10     e23     V11     e20     


        e24             e25             e26             e27     


e28     V12     e29     V13     e30     V14     e31     V15     e28     


        e0              e1              e2              e3      




D = 4 repeating graph



The eagle-eyed will notice the same vertices would be flipped if edges `e22` and `e23` were flipped. In fact, there are quite a few edge chains that would do this, for example:

In [71]:
edges_to_flip = [5, 6, 7, 8, 11, 13, 17, 19]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)


        e0              e1              e2              e3      


e4      V0      e5      V1      e6      V2      e7      V3      e4      


        e8              e9              e10             e11     


e12     V4      e13     V5      e14     V6      e15     V7      e12     


        e16             e17             e18             e19     


e20     V8      e21     V9      e22     V10     e23     V11     e20     


        e24             e25             e26             e27     


e28     V12     e29     V13     e30     V14     e31     V15     e28     


        e0              e1              e2              e3      




D = 4 repeating graph



What happens when an edge chain goes from end-to-end either top/bottom or left/right? Let's see:

In [72]:
edges_to_flip = [1, 9, 14, 18, 22, 25]
flipped_vertices = graph.mark_vertices(edges_to_flip)

graph.draw_graph(edges_to_flip, flipped_vertices)


        e0              e1              e2              e3      


e4      V0      e5      V1      e6      V2      e7      V3      e4      


        e8              e9              e10             e11     


e12     V4      e13     V5      e14     V6      e15     V7      e12     


        e16             e17             e18             e19     


e20     V8      e21     V9      e22     V10     e23     V11     e20     


        e24             e25             e26             e27     


e28     V12     e29     V13     e30     V14     e31     V15     e28     


        e0              e1              e2              e3      




D = 4 repeating graph



No vertices are in the `1` state!